# Closed-form solution

In [1]:
%display latex

# https://github.com/jupyter-lsp/jupyterlab-lsp/issues/910#issuecomment-1465356625

In [2]:
l = var("l", latex_name="\\lambda")

H = matrix(
    SR,
    [
        [var(f"R_{j}{i}" if i < 3 else f"t_{j}") for i in range(1, 4)]
        for j in range(1, 4)
    ],
)
v1 = vector(H[:, 0]) * l
v2 = vector(H[:, 1]) * l

(v1, v2)

((R_11*l, R_21*l, R_31*l), (R_12*l, R_22*l, R_32*l))

# Eq 10

In [3]:
eqs = [v1 * v2 == 0, norm(v1) == 1, norm(v2) == 1]

eqs

[R_11*R_12*l^2 + R_21*R_22*l^2 + R_31*R_32*l^2 == 0,
 sqrt(abs(R_11*l)^2 + abs(R_21*l)^2 + abs(R_31*l)^2) == 1,
 sqrt(abs(R_12*l)^2 + abs(R_22*l)^2 + abs(R_32*l)^2) == 1]

In [4]:
assume(l, "real")
for el in H.list():
    for v in el.variables():
        assume(v, "real")

In [5]:
[eq.simplify_full() for eq in eqs]

[(R_11*R_12 + R_21*R_22 + R_31*R_32)*l^2 == 0,
 sqrt(R_11^2 + R_21^2 + R_31^2)*abs(l) == 1,
 sqrt(R_12^2 + R_22^2 + R_32^2)*abs(l) == 1]

In [6]:
print(eqs)

[R_11*R_12*l^2 + R_21*R_22*l^2 + R_31*R_32*l^2 == 0, sqrt((R_11^2*l^2) + (R_21^2*l^2) + (R_31^2*l^2)) == 1, sqrt((R_12^2*l^2) + (R_22^2*l^2) + (R_32^2*l^2)) == 1]


We set f1 and f2 to 1, since we could have incorporated it into H before.

In [25]:
solution = solve(eqs, [v1[2]/l, v2[2]/l])


KeyboardInterrupt



In [8]:
len(solution)

8

In [9]:
# for i, sol in enumerate(solution, 1):
#     print('[')
#     for s in sol:
#         print(str(s.simplify_full().rhs()).replace('R_', 'r_').replace('sqrt', '√') + ',')
#     print('],')

# Scaramuzza solver

## Given equations

In [10]:
v1, v2

((R_11*l, R_21*l, R_31*l), (R_12*l, R_22*l, R_32*l))

In [35]:
eq1, eq2, eq3 = eqs

for e in (eq1, eq2, eq3):
    display(e)

R_11*R_12*l^2 + R_21*R_22*l^2 + R_31*R_32*l^2 == 0

sqrt((R_11^2*l^2) + (R_21^2*l^2) + (R_31^2*l^2)) == 1

sqrt((R_12^2*l^2) + (R_22^2*l^2) + (R_32^2*l^2)) == 1

## Equation from the matlab code:

In [12]:
AA = (v1[0] * v2[0] + v1[1] * v2[1]) ^ 2
BB = v1[0] ^ 2 + v1[1] ^ 2
CC = v2[0] ^ 2 + v2[1] ^ 2

for e in (AA, BB, CC):
    display(e)

(R_11*R_12*l^2 + R_21*R_22*l^2)^2

R_11^2*l^2 + R_21^2*l^2

R_12^2*l^2 + R_22^2*l^2

In [13]:
scar_eq = 1 * v2[2] ^ 4 + (CC - BB) * v2[2] ^ 2 - AA == 0
scar_eq

R_32^4*l^4 - (R_11^2*l^2 - R_12^2*l^2 + R_21^2*l^2 - R_22^2*l^2)*R_32^2*l^2 - (R_11*R_12*l^2 + R_21*R_22*l^2)^2 == 0

## Derivation

In [14]:
sols = eq1.solve(v1[2]/l)
assert len(sols) == 1
sol = sols[0]
sol

R_31 == -(R_11*R_12 + R_21*R_22)/R_32

In [15]:
e1 = eq2.subs(sol)
e1

sqrt((R_11*R_12 + R_21*R_22)^2*l^2/R_32^2 + (R_11^2*l^2) + (R_21^2*l^2)) == 1

In [16]:
e1 ^ 2

(R_11*R_12 + R_21*R_22)^2*l^2/R_32^2 + (R_11^2*l^2) + (R_21^2*l^2) == 1

In [17]:
e2 = e1 ^ 2 - eq3 ^ 2
e2

(R_11*R_12 + R_21*R_22)^2*l^2/R_32^2 + (R_11^2*l^2) - (R_12^2*l^2) + (R_21^2*l^2) - (R_22^2*l^2) - (R_32^2*l^2) == 0

In [18]:
e3 = (e2 * v2[2] ^ 2).simplify_full()

In [19]:
assert e3 == -1 * scar_eq.expand().simplify_full()

In [20]:
for s in solve(e3, v2[2]/l):
    display(s ^ 2)
    display((sol.subs(s)).simplify_full())

R_32^2 == 1/2*R_11^2 - 1/2*R_12^2 + 1/2*R_21^2 - 1/2*R_22^2 + 1/2*sqrt(R_11^4 + 2*R_11^2*R_12^2 + R_12^4 + 2*R_11^2*R_21^2 - 2*R_12^2*R_21^2 + R_21^4 + 8*R_11*R_12*R_21*R_22 - 2*R_11^2*R_22^2 + 2*R_12^2*R_22^2 + 2*R_21^2*R_22^2 + R_22^4)

R_31 == (R_11*R_12 + R_21*R_22)/sqrt(1/2*R_11^2 - 1/2*R_12^2 + 1/2*R_21^2 - 1/2*R_22^2 + 1/2*sqrt(R_11^4 + 2*R_11^2*R_12^2 + R_12^4 + R_21^4 + 8*R_11*R_12*R_21*R_22 + R_22^4 + 2*(R_11^2 - R_12^2)*R_21^2 - 2*(R_11^2 - R_12^2 - R_21^2)*R_22^2))

R_32^2 == 1/2*R_11^2 - 1/2*R_12^2 + 1/2*R_21^2 - 1/2*R_22^2 + 1/2*sqrt(R_11^4 + 2*R_11^2*R_12^2 + R_12^4 + 2*R_11^2*R_21^2 - 2*R_12^2*R_21^2 + R_21^4 + 8*R_11*R_12*R_21*R_22 - 2*R_11^2*R_22^2 + 2*R_12^2*R_22^2 + 2*R_21^2*R_22^2 + R_22^4)

R_31 == -(R_11*R_12 + R_21*R_22)/sqrt(1/2*R_11^2 - 1/2*R_12^2 + 1/2*R_21^2 - 1/2*R_22^2 + 1/2*sqrt(R_11^4 + 2*R_11^2*R_12^2 + R_12^4 + R_21^4 + 8*R_11*R_12*R_21*R_22 + R_22^4 + 2*(R_11^2 - R_12^2)*R_21^2 - 2*(R_11^2 - R_12^2 - R_21^2)*R_22^2))

R_32^2 == 1/2*R_11^2 - 1/2*R_12^2 + 1/2*R_21^2 - 1/2*R_22^2 - 1/2*sqrt(R_11^4 + 2*R_11^2*R_12^2 + R_12^4 + 2*R_11^2*R_21^2 - 2*R_12^2*R_21^2 + R_21^4 + 8*R_11*R_12*R_21*R_22 - 2*R_11^2*R_22^2 + 2*R_12^2*R_22^2 + 2*R_21^2*R_22^2 + R_22^4)

R_31 == (R_11*R_12 + R_21*R_22)/sqrt(1/2*R_11^2 - 1/2*R_12^2 + 1/2*R_21^2 - 1/2*R_22^2 - 1/2*sqrt(R_11^4 + 2*R_11^2*R_12^2 + R_12^4 + R_21^4 + 8*R_11*R_12*R_21*R_22 + R_22^4 + 2*(R_11^2 - R_12^2)*R_21^2 - 2*(R_11^2 - R_12^2 - R_21^2)*R_22^2))

R_32^2 == 1/2*R_11^2 - 1/2*R_12^2 + 1/2*R_21^2 - 1/2*R_22^2 - 1/2*sqrt(R_11^4 + 2*R_11^2*R_12^2 + R_12^4 + 2*R_11^2*R_21^2 - 2*R_12^2*R_21^2 + R_21^4 + 8*R_11*R_12*R_21*R_22 - 2*R_11^2*R_22^2 + 2*R_12^2*R_22^2 + 2*R_21^2*R_22^2 + R_22^4)

R_31 == -(R_11*R_12 + R_21*R_22)/sqrt(1/2*R_11^2 - 1/2*R_12^2 + 1/2*R_21^2 - 1/2*R_22^2 - 1/2*sqrt(R_11^4 + 2*R_11^2*R_12^2 + R_12^4 + R_21^4 + 8*R_11*R_12*R_21*R_22 + R_22^4 + 2*(R_11^2 - R_12^2)*R_21^2 - 2*(R_11^2 - R_12^2 - R_21^2)*R_22^2))

In [23]:
for s in solve(e3, v2[2]/l):
    print(s ^ 2)

R_32^2 == 1/2*R_11^2 - 1/2*R_12^2 + 1/2*R_21^2 - 1/2*R_22^2 + 1/2*sqrt(R_11^4 + 2*R_11^2*R_12^2 + R_12^4 + 2*R_11^2*R_21^2 - 2*R_12^2*R_21^2 + R_21^4 + 8*R_11*R_12*R_21*R_22 - 2*R_11^2*R_22^2 + 2*R_12^2*R_22^2 + 2*R_21^2*R_22^2 + R_22^4)
R_32^2 == 1/2*R_11^2 - 1/2*R_12^2 + 1/2*R_21^2 - 1/2*R_22^2 + 1/2*sqrt(R_11^4 + 2*R_11^2*R_12^2 + R_12^4 + 2*R_11^2*R_21^2 - 2*R_12^2*R_21^2 + R_21^4 + 8*R_11*R_12*R_21*R_22 - 2*R_11^2*R_22^2 + 2*R_12^2*R_22^2 + 2*R_21^2*R_22^2 + R_22^4)
R_32^2 == 1/2*R_11^2 - 1/2*R_12^2 + 1/2*R_21^2 - 1/2*R_22^2 - 1/2*sqrt(R_11^4 + 2*R_11^2*R_12^2 + R_12^4 + 2*R_11^2*R_21^2 - 2*R_12^2*R_21^2 + R_21^4 + 8*R_11*R_12*R_21*R_22 - 2*R_11^2*R_22^2 + 2*R_12^2*R_22^2 + 2*R_21^2*R_22^2 + R_22^4)
R_32^2 == 1/2*R_11^2 - 1/2*R_12^2 + 1/2*R_21^2 - 1/2*R_22^2 - 1/2*sqrt(R_11^4 + 2*R_11^2*R_12^2 + R_12^4 + 2*R_11^2*R_21^2 - 2*R_12^2*R_21^2 + R_21^4 + 8*R_11*R_12*R_21*R_22 - 2*R_11^2*R_22^2 + 2*R_12^2*R_22^2 + 2*R_21^2*R_22^2 + R_22^4)


In [24]:
for s in solve(e3.subs(l == 1), v2[2]/l):
    print(s ^ 2)

R_32^2 == 1/2*R_11^2 - 1/2*R_12^2 + 1/2*R_21^2 - 1/2*R_22^2 + 1/2*sqrt(R_11^4 + 2*R_11^2*R_12^2 + R_12^4 + 2*R_11^2*R_21^2 - 2*R_12^2*R_21^2 + R_21^4 + 8*R_11*R_12*R_21*R_22 - 2*R_11^2*R_22^2 + 2*R_12^2*R_22^2 + 2*R_21^2*R_22^2 + R_22^4)
R_32^2 == 1/2*R_11^2 - 1/2*R_12^2 + 1/2*R_21^2 - 1/2*R_22^2 + 1/2*sqrt(R_11^4 + 2*R_11^2*R_12^2 + R_12^4 + 2*R_11^2*R_21^2 - 2*R_12^2*R_21^2 + R_21^4 + 8*R_11*R_12*R_21*R_22 - 2*R_11^2*R_22^2 + 2*R_12^2*R_22^2 + 2*R_21^2*R_22^2 + R_22^4)
R_32^2 == 1/2*R_11^2 - 1/2*R_12^2 + 1/2*R_21^2 - 1/2*R_22^2 - 1/2*sqrt(R_11^4 + 2*R_11^2*R_12^2 + R_12^4 + 2*R_11^2*R_21^2 - 2*R_12^2*R_21^2 + R_21^4 + 8*R_11*R_12*R_21*R_22 - 2*R_11^2*R_22^2 + 2*R_12^2*R_22^2 + 2*R_21^2*R_22^2 + R_22^4)
R_32^2 == 1/2*R_11^2 - 1/2*R_12^2 + 1/2*R_21^2 - 1/2*R_22^2 - 1/2*sqrt(R_11^4 + 2*R_11^2*R_12^2 + R_12^4 + 2*R_11^2*R_21^2 - 2*R_12^2*R_21^2 + R_21^4 + 8*R_11*R_12*R_21*R_22 - 2*R_11^2*R_22^2 + 2*R_12^2*R_22^2 + 2*R_21^2*R_22^2 + R_22^4)
